In [0]:
import re
import csv
from sklearn.naive_bayes import GaussianNB
from string import punctuation
import nltk 
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [0]:
f1 = '/data_task1/train_filled.json'
f2='/data_task1/test_filled.json'
f3='/data_task1/test_china.json'

In [0]:
pd.options.display.max_colwidth =5000

df1= pd.read_json(f1,lines=True)
df1.head(34)
df2= pd.read_json(f2,lines=True)
df2.head(34)
df3= pd.read_json(f3, lines=True)
#df3.head(34)

In [0]:
news = df1["text"].tolist()
labels =df1["label"].tolist()
news_test=df2["text"].tolist()
#labels_test=df2["label"].tolist()
#print (news[0])
#print (news_test[0])
print (labels)

[0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 

In [0]:
id = df1["id"].tolist()
id_test= df2["id"].tolist()
id_test_true= df3["id"].tolist()

print(len(id))
print(len(id_test))
print(len(id_test_true))

3430
687
2303


# Preprocessing 

### Cleaning data

In [0]:
stop_words = stopwords.words('english')
#print (stop_words)

def preprocess(new):
    #Remove html tags
    new= re.sub (r'<.*?>',' ',new)
    #Remove punctuation 
    new = re.sub('[^\w\s]',' ',new)
    #Remove numbers and  strings starting with numbers
    new = re.sub(r'\s*\d\w*',' ',new)
    #Remove string in capital
    #new= re.sub('\s[A-Z]+\w*',' ',new)
    #Remove additional white spaces
    new = re.sub('\s+', ' ', new)
    return (new)

def word_extraction(new):
    words = new.split(" ")
    #print (words)
    splitted_text = [w for w in words if w !=""]
    return splitted_text

  

In [0]:
#import truecase
#prp_news=  [preprocess(new)for new in news ]
#prp_news_test=  [preprocess(new)for new in news_test ]
#print (prp_news[0])
#news_tc=[truecase(new) for new in news]
#newstest_tc=[truecase(new) for new in news_test]
#word_news= [word_extraction(preprocess(truecase.get_true_case(new)))for new in news ]
#word_news_test= [  word_extraction(preprocess(truecase.get_true_case(new))) for new in news_test ]

#####******* Module truecase not find in Google Colaboratory, data were computed externally and then imported******

t1 = '/data_task1/truecase.txt'
t2='/data_task1/truecasetest.txt'
t3='/data_task1/truecasetestchina.txt'

with open(t1) as f:
  word_news= [word_extraction(preprocess(new))for new in f.readlines()]

print (word_news[0:4])
print (len(word_news))

with open(t2) as f:
  word_news_test= [word_extraction(preprocess(new))for new in f.readlines()]

print (word_news_test[0:4])
print (len(word_news_test))

[['Education', 'must', 'for', 'Tribals', 'Bhubaneswar', 'describing', 'education', 'as', 'essential', 'for', 'development', 'of', 'the', 'downtrodden', 'especially', 'of', 'the', 'tribal', 'community', 'justice', 'Dalveer', 'Bhandari', 'judge', 'in', 'international', 'Court', 'of', 'justice', 'said', 'it', 'can', 'help', 'them', 'join', 'mainstream', 'society', 'education', 'gives', 'knowledge', 'along', 'with', 'social', 'security', 'and', 'success', 'in', 'life', 'justice', 'Bhandari', 'said', 'during', 'a', 'visit', 'to', 'Kiit', 'and', 'kiss', 'here', 'recently', 'advising', 'entrepreneurs', 'to', 'invest', 'more', 'in', 'building', 'learning', 'centres', 'he', 'said', 'reputed', 'universities', 'like', 'Oxford', 'Cambridge', 'and', 'Stanford', 'are', 'all', 'set', 'up', 'by', 'entrepreneurs', 'or', 'private', 'trust', 'boards', 'Justice', 'Bhandari', 'received', 'doctorate', 'in', 'law', 'honorary', 'Causa', 'which', 'was', 'conferred', 'on', 'him', 'during', 'the', 'Convocation',

In [0]:
sw1 = [line.rstrip('\n') for line in open("/data_task1/rmstop.txt")]
sw2=[line.rstrip('\n') for line in open('/data_task1/smart_common_words.txt")]
print(len(sw1))
print(len(sw2))
stop_words = stopwords.words('english')
print (stop_words)
stop_words= stop_words + sw1 +sw2
stop_words.extend(["ist"])
print(len(stop_words))
stop_words= list(set(stop_words)) 
print(sw2[:32])

566
617
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', '

### NER

In [0]:
import nltk
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.chunk import conlltags2tree, tree2conlltags
from pprint import pprint

nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('maxent_ne_chunker')


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.


'from nltk.tag.stanford import NERTagger\nfrom nltk.tag import StanfordNERTagger\nst = StanfordNERTagger(\'stanford-ner-tagger/english.all.3class.distsim.crf.ser.gz\',\'stanford-ner-tagger/stanford-ner.jar\')\nsentences=truecase_news\nprint (len(sentences))\nnner_news=[]\n    \nfor i in sentences:\n    for sent in nltk.sent_tokenize(i):\n        tokens = nltk.tokenize.word_tokenize(sent)\n        print i\n        #print (tokens)µ\n        tags = st.tag(tokens)\n        #print (tags)\n    t=[i[0] for i in tags if i[1] not in ["PERSON", "LOCATION", "ORGANIZATION"]]\n    ner_news.append(t)'

In [0]:
#ner = [line.split() for line.strip in open("ner.txt")]
import ast
ner= [ast.literal_eval(line.rstrip('\n') )for line in  open("/data_task1/ner.txt")]
ner1= [ast.literal_eval(line.rstrip('\n') )for line in  open("/data_task1/ner1.txt")]
#ner1= [line for line in open("ner1.txt")]
ner_news= ner + ner1[1:]
print (ner_news[0:3])
print (len(ner_news))

[['Education', 'must', 'for', 'describing', 'education', 'as', 'essential', 'for', 'development', 'of', 'the', 'downtrodden', 'especially', 'of', 'the', 'tribal', 'community', 'justice', 'judge', 'in', 'international', 'Court', 'of', 'justice', 'said', 'it', 'can', 'help', 'them', 'join', 'mainstream', 'society', 'education', 'gives', 'knowledge', 'along', 'with', 'social', 'security', 'and', 'success', 'in', 'life', 'justice', 'said', 'during', 'a', 'visit', 'to', 'and', 'kiss', 'here', 'recently', 'advising', 'entrepreneurs', 'to', 'invest', 'more', 'in', 'building', 'learning', 'centres', 'he', 'said', 'reputed', 'universities', 'like', 'and', 'are', 'all', 'set', 'up', 'by', 'entrepreneurs', 'or', 'private', 'trust', 'boards', 'justice', 'received', 'doctorate', 'in', 'law', 'honorary', 'Causa', 'which', 'was', 'conferred', 'on', 'him', 'during', 'the', 'Convocation', 'of', 'conferred', 'D', 'law', 'on', 'justice', 'in', 'the', 'presence', 'of', 'Dr', 'founder', 'of', 'and', 'kiss'

In [0]:
ner_news_test= [ast.literal_eval(line.rstrip('\n') )for line in  open("/data_task1/nertest.txt")]
print (len(ner_news_test))

687


### POS TAGGING

In [0]:
postag=[nltk.pos_tag(i)for i in ner_news]
postag_test= [nltk.pos_tag(i)for i in ner_news_test]
print (len(postag_test))

687


Scenario 1

In [0]:
tagged_news=[]
for i in postag:
    l=[]
    l=[ x for (x,y) in i if   (  y.startswith('VB')  or y.startswith('RB') )  and  x not in stop_words]         
    tagged_news.append(l)
    
tagged_news_test=[]
for i in postag_test:
    l=[]
    l= [ x for (x,y) in i if (  y.startswith('VB')  or y.startswith('RB') ) and  x not in stop_words ] 
    tagged_news_test.append(l)


Scenario 2

In [0]:
tagged_news=[]
for i in postag:
    l=[]
    l=[ x for (x,y) in i if   (y=="NNS" or y=="NN" or  y.startswith('VB')  or y.startswith('RB') or y.startswith('JJ')) and  x not in stop_words   ]         
    tagged_news.append(l)
    
tagged_news_test=[]
for i in postag_test:
    l=[]
    l= [ x for (x,y) in i if ( y=="NNS" or y=="NN" or  y.startswith('VB')  or y.startswith('RB') or y.startswith('JJ')) and  x not in stop_words  ] 
    tagged_news_test.append(l)
    
print (tagged_news_test[0])


['Top', 'searches', 'man', 'wife', 'assaulted', 'dowry', 'woman', 'allegedly', 'Chained', 'assaulted', 'laws', 'demanded', 'dowry', 'woman', 'admitted', 'hospital', 'severe', 'injuries', 'victim', 'identified', 'reportedly', 'locked', 'room', 'tip', 'police', 'reached', 'spot', 'rushed', 'hospital', 'police', 'source', 'revealed', 'married', 'employee', 'municipal', 'corporation', 'recently', 'brought', 'parents', 'month', 'Pestered', 'laws', 'ordered', 'bring', 'source', 'added', 'expressed', 'helplessness', 'bring', 'money', 'parents', 'poor', 'people', 'children', 'locked', 'room', 'police', 'investigation', 'bail', 'smugglers', 'additional', 'sessions', 'granted', 'bail', 'village', 'allegedly', 'involved', 'sandalwood', 'smuggling', 'national', 'highway', 'recalled', 'accomplice', 'allegedly', 'carrying', 'sandalwood', 'logs', 'tempo', 'allegedly', 'assaulted', 'Constable', 'iron', 'rods', 'pistol', 'rifle', 'gang', 'fired', 'rounds', 'rifle', 'squad', 'national', 'highway', 'poli

In [0]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
preprocessed_news= [ TreebankWordDetokenizer().detokenize(i) for i in tagged_news]
print(preprocessed_news[0:3])
from nltk.tokenize.treebank import TreebankWordDetokenizer
preprocessed_news_test= [ TreebankWordDetokenizer().detokenize(i) for i in tagged_news_test]
print(len(preprocessed_news))
print(len(preprocessed_news_test))


['describing join knowledge kiss recently advising invest building set received conferred conferred kiss', 'Bridging show reconstruct ravaged lost talking reveals Drape shawl sound stands earn means close worked working exposed chicken work voluntarily centre works assist develop giving required intends bridge visited', 'expecting failed deliver doled Moreover contend held made demanding sensing laid asked doubling acknowledging give performing Laggards sensed poised grow wanted treated prepared face reiterating customs brought gulp appeared protesting bringing level urged faced continue welcomed desperately indirectly benefit asked enhancing revive oriented intensify reform regretted lecture']
3430
687


# Feature extraction

### Bag of Words : TF-IDF

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

In [0]:
stemmer = SnowballStemmer('english', ignore_stopwords=False)

class StemmedTfidfVectorizer(TfidfVectorizer):
    
    def __init__(self, stemmer, *args, **kwargs):
        super(StemmedTfidfVectorizer, self).__init__(*args, **kwargs)
        self.stemmer = stemmer
        
    def build_analyzer(self):
        analyzer = super(StemmedTfidfVectorizer, self).build_analyzer()
        return lambda doc: (self.stemmer.stem(word) for word in analyzer(doc.replace('\n', ' ')))

In [0]:
#stemming and unigram
vectorizer_stem_u = StemmedTfidfVectorizer(stemmer=stemmer, sublinear_tf=True)
X_train_stem_u = vectorizer_stem_u.fit_transform(preprocessed_news)
print(X_train_stem_u.shape)

tf1 = vectorizer_stem_u.get_feature_names()
print (tf1[0:200])

(3430, 5432)
['__', 'aback', 'abandon', 'abat', 'abbrevi', 'abdic', 'abdomen', 'abduct', 'abet', 'abid', 'abl', 'ablaz', 'abnorm', 'abolish', 'abort', 'abound', 'abreast', 'abroad', 'abrog', 'abrupt', 'abscond', 'absent', 'absolut', 'absolv', 'absond', 'absorb', 'abstain', 'abund', 'abus', 'abut', 'abysm', 'academ', 'acced', 'acceler', 'accent', 'accentu', 'accept', 'accepteth', 'access', 'accessori', 'accid', 'accident', 'acclaim', 'acclimatis', 'accommod', 'accompani', 'accomplish', 'accord', 'accost', 'account', 'accredit', 'accru', 'accumul', 'accur', 'accus', 'accustom', 'ach', 'acheiv', 'achiev', 'acid', 'acknowledg', 'acquaint', 'acquir', 'acquit', 'acr', 'act', 'activ', 'activist', 'actor', 'actress', 'acut', 'ad', 'adam', 'adapt', 'add', 'addict', 'addit', 'address', 'adept', 'adequ', 'adher', 'adjac', 'adjoin', 'adjourn', 'adjud', 'adjudg', 'adjust', 'administ', 'administr', 'admir', 'admiss', 'admit', 'admonish', 'adopt', 'ador', 'adorn', 'adulter', 'advanc', 'advent', 'adve

In [0]:
X_train_stem_u_test = vectorizer_stem_u.transform(preprocessed_news_test)
print(X_train_stem_u_test.shape)

(687, 5432)


In [0]:
##stemming and bigrams
vectorizer_stem_b = StemmedTfidfVectorizer(stemmer=stemmer, ngram_range=(1,2), sublinear_tf=True)
X_train_stem_b = vectorizer_stem_b.fit_transform(preprocessed_news)
print(X_train_stem_b.shape)
tf2 = vectorizer_stem_b.get_feature_names()
print (tf2[0:200])

(3430, 90873)
['__', '__ __', '__ accomplish', '__ develop', '__ dress', '__ let', '__ outliv', '__ rais', '__ report', '__ sever', '__ take', 'aback', 'aback hit', 'abandon', 'abandon issu', 'abandon recov', 'abandon sel', 'abandon set', 'abandon tak', 'abandon ultim', 'abandoned abandon', 'abandoned ad', 'abandoned appear', 'abandoned build', 'abandoned dump', 'abandoned escap', 'abandoned examin', 'abandoned fil', 'abandoned inspect', 'abandoned kadookkoop', 'abandoned ref', 'abandoned remain', 'abandoned shock', 'abandoned slight', 'abandoned start', 'abandoned suspect', 'abandoned yield', 'abandoning criticis', 'abandoning deposit', 'abandoning presid', 'abandoning stand', 'abandoning view', 'abandoning vow', 'abat', 'abated signific', 'abating record', 'abbrevi', 'abbreviated stem', 'abdic', 'abdicate pass', 'abdomen', 'abdomen show', 'abduct', 'abducted accus', 'abducted convey', 'abducted forc', 'abducted hit', 'abducted interf', 'abducted sleep', 'abducted stop', 'abducting hi

In [0]:
X_train_stem_b_test = vectorizer_stem_b.transform(preprocessed_news_test)
print(X_train_stem_b_test.shape)

(687, 90873)


# CLASSIFIER

### Naives Bayes Classifier

#### Test set = 20% training set

In [0]:
#testing on splitted training set
from sklearn.model_selection import train_test_split  
from sklearn.naive_bayes import GaussianNB

X_train, X_test, y_train, y_test = train_test_split(X_train_stem_b, labels, test_size=0.2, random_state=0)  
# fit the training dataset on the NB classifier
Naive = naive_bayes.GaussianNB()
Naive.partial_fit(X_train.toarray(),y_train, classes=[0,1])
print(Naive)
# predict the labels on validation dataset
predict_NB = Naive.predict(X_test.toarray())
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predict_NB, y_test)*100)

from sklearn.metrics import f1_score
score = f1_score(predict_NB , y_test)

# print
print (score)

GaussianNB(priors=None, var_smoothing=1e-09)
Naive Bayes Accuracy Score ->  79.44606413994168
0.41975308641975306


#### Test set = test_filled.json

In [0]:
# fit the training dataset on the NB classifier


# fit the training dataset on the NB classi
Naive = naive_bayes.GaussianNB()
Naive.partial_fit(X_train_stem_b.toarray(),labels,classes=[0,1])
print(Naive)
# predict the labels on validation dataset
predictions_NB = Naive.predict(X_train_stem_b_test.toarray())
print (predictions_NB)

GaussianNB(priors=None, var_smoothing=1e-09)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 1 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 1 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1
 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1
 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0
 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0
 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 

'Naive = naive_bayes.MultinomialNB()\nNaive.fit(X_train_stem_b,labels)\nprint(Naive)\n# predict the labels on validation dataset\npredictions_NB = Naive.predict(X_train_stem_b_test)\nprint (predictions_NB'

### SVM

#### Test set = 20% training set

In [0]:
#testing on splitted training set
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X_train_stem_b, labels, test_size=0.2, random_state=0) 
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train,y_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(X_test)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test)*100)

from sklearn.metrics import f1_score
score = f1_score(predictions_SVM , y_test)

# print
print (score)

SVM Accuracy Score ->  86.58892128279884
0.5779816513761468


In [0]:
print(len(predictions_SVM))

686


#### Test set = test_filled.json

In [0]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_stem_b,labels)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(X_train_stem_b_test)
print (len(predictions_SVM))

687


In [0]:
print (predictions_SVM)

[0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0
 1 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 0 0
 0 0 0 0 1 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0
 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 0
 0 0 1 1 0 0 0 0 1 0 1 1 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0
 0 0 0 1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1
 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 0 

In [0]:
nb_protest_SVM=sum([1 for x in predictions_SVM if x ==1])
print(nb_protest_SVM)
p = pd.DataFrame(
    {'new': tagged_news_test,
     'label': predictions_SVM
    })
pd.options.display.max_colwidth =5000
from IPython.core.display import HTML
display(HTML(p.to_html()))

### Random Forest

#### Test set  = 20% training set

In [0]:
#testing on splitted training set
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X_train_stem_b, labels, test_size=0.2, random_state=0) 
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)  
classifier.fit(X_train,y_train)
predictions_RF= classifier.predict(X_test)  


In [0]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(predictions_RF,y_test))   
print(accuracy_score(predictions_RF,y_test))
from sklearn.metrics import f1_score
score = f1_score(predictions_RF, y_test)

# print
print (score)

[[548 111]
 [  5  22]]
0.8309037900874635
0.275


#### Test set = test_filled.json

In [0]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)  
classifier.fit(X_train_stem_b,labels)
predictions_RF= classifier.predict(X_train_stem_b_test)  

In [0]:
nb_protest_RF=sum([1 for x in predictions_RF if x ==1])
print(nb_protest_RF)
print(predictions_RF)

47
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0
 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0

## Random Forest vs SVM

In [0]:
  import pandas as pd
  df = pd.DataFrame({'Random Forest': [], 'SVM': [], 'news': []})


for i, (a, b,c) in enumerate(zip(predictions_RF,
 predictions_SVM,news_test)):    
    if a!=b:
        df = df.append({'Random Forest': a, 'SVM': b, 'news': c}, ignore_index=True)
print (df.shape)        
pd.options.display.max_colwidth =5000
from IPython.core.display import HTML
display(HTML(df.to_html()))

(101, 3)


,Random Forest,SVM,news
0,0.0,1.0,"Teachers strike disrupts studies | Ahmedabad News - Times of India\nTNN | Sep 3, 2001, 23:55 IST\nTeachers strike disrupts studies\nTNN | Sep 3, 2001, 23:55 IST\nahmedabad: how justified are the teachers in holding the entire education system to ransom for a vehicle allowance of rs 65?, asks a jagrut vali mandal hand-out that thousands of parents in city received on monday morning, hidden within folds of their newspapers. late it may be, but the hand-bill has come as a wake-up call to parents of secondary and higher secondary students who are on a forced vacation for one week, all thanks to the striking teachers and principals. almost 70 per cent of the schools were scheduled to conduct the first tests from monday but the same has been indefinitely postponed. ""come to think of it, the strike by teachers is highly unjustified. our children have already lost almost four months of education because of the earthquake, and now this strike. and to put it frankly, their demands are too petty to put education of 30 lakh students on hold,"" says an agitated parent of a class x student of firdaus amrut high school, cantonment. the government has already partially responded to the state-wide school bandh by issuing a gr ordering 100 per cent recruitment of teachers and also announcing vehicle allowance from january 1, 2001. the two were proclaimed as the main demands when the teachers had embarked on the indefinite strike. ""apparently now, they want to extract more by pressuring the government through the strike. but should our children suffer? of 365 days in the year, the teachers are entitled to leave of around 146 days. add to this, the 125 days that were lost in earthquake, the total working days come to around 94 days. do the teachers not have the moral responsibility to ensure that the students study at least 94 days?"" questions father of a class xi student of nr higher secondary school. here, while striking teachers and principals concede that there is no way they will resume teaching until all of their six-demands are met -- those include giving teachers vehicle allowance with effect from april 1, 1998, and other demands of administrative and financial implications -- they assured that each day of education lost due to the strike will be covered up. ""once the strike is called off, we will teach on sunday, public holidays and if need be even shorten the diwali vacation and ensure that the loss is covered up,"" assured president of the gujarat rajya shaikshanik sankalan samiti keshubahi patel. but just when the teachers will end the strike, no one knows. ""it is going to take long ... this is what our experience says,"" says patel. ""till then, we just kill time. when their issues get resolved, we will be forced to take these extra-classes on sundays and public holidays during diwali? and they teach us selflessness is class, what a sham,"" says a ssc board student."
1,0.0,1.0,"Committee Drops Inspection at Mullaiperiyar Dam\n2014-12-17 06:01:00\nCUMBUM: The five-member sub-committee constituted by the Supreme Court-appointed supervisory panel cancelled the inspection of the Mullaiperiyar dam after Kerala forest officials allegedly asked the Tamil Nadu members of the panel to produce their identity cards and sign their register to visit the dam in Thekkady on Tuesday, a senior Tamil Nadu PWD officer has said.\nThe sub-committee headed by Central Water Commission Executive Engineer Umberji Harish Girish, along with Tamil Nadu representatives — PWD Mullaiperiyar dam Executive Engineer A Madhavan, Assistant Executive Engineer A Soundaram--and Kerala Irrigation Department Executive Engineer George Daniel and Assistant Engineer N S Praseed, assembled at the boat jetty in Thekkady to visit the dam.\n“But Kerala forest officials demanded to produce our identity cards and sign their register to board the boat,” the senior PWD officer said. However, refusing to sign the register, the TN representatives 